In [1]:
import numpy as np
import numpy.linalg as LA
import sympy as sm

## Метод колокації для інтегрального рівняння Фредгольма другого роду

Маємо інтегральне рівняння вигляду

$$
\varphi(x)-\lambda \int_{a}^{b} \varphi(y) K(x, y) d y = f(x), \quad x \in[a, b]
$$

Наближений розв'язок будемо шукати у вигляді

$$
\tilde{\varphi}(x)=\sum_{j=1}^{n} c_{j} \gamma_{j}(x)
$$

де $c_j$ - невідомі константи, які потрібно знайти, а $\gamma_j$ - задані лінійнонезалежні функції (в нашому випадку кусково аналітичні), $j = 1, ..., n$.

Тоді отримаємо такий вираз

$$
\sum_{j=1}^{n} c_{j}\left[\gamma_{j}(x)-\lambda \int_{a}^{b} K(x, y) \gamma_{j}(y) d y\right]=f(x)
$$

Розглянувши його на множині точок $x_i$, таких, що
$$
a \leqslant x_{1}<x_{2}<\ldots<x_{m} \leqslant b
$$

отримаємо СЛАР відносно невідомих $с_j$, $j = 1, ..., n$. 

In [2]:
x, y = sm.symbols('x y')

In [3]:
a, b = -1, 1
K = lambda x_, y_: x_*y_ + x_**2
f = sm.S.One
lambd = 1
h = 0.1

In [4]:
xx = np.arange(a, b + 0.000001, (b-a)*h)
n = len(xx)
A = np.zeros((n,n))

In [5]:
xx.round(4)

array([-1. , -0.8, -0.6, -0.4, -0.2, -0. ,  0.2,  0.4,  0.6,  0.8,  1. ])

In [6]:
def basicFunction(j, x, xx=xx, h=h):
  for x_ in
  1 + (x - xx[j])/h

SyntaxError: invalid syntax (<ipython-input-6-db337b88dd20>, line 2)

In [ ]:
for i in range(n):
  for j in range(n):
    integral = sm.integrate(K(xx[i], y)*basicFunction(j, y), (y, a, b))
    print(integral)
    #print((basicFunction(j, x) - integral).evalf(5,subs={x: xx[i]}))
    A[i,j] = (basicFunction(j, x) - integral).evalf(5,subs={x: xx[i]})

A

In [ ]:
ff = [[f.evalf(5, subs={x: xx[j]})] for j in range(n)]
ff

In [ ]:
A = np.array(A, dtype='float')
A.shape

In [ ]:
ff = np.array(ff, dtype='float')
ff.shape

In [ ]:
c = LA.solve(A, ff) 

In [ ]:
c.shape

In [ ]:
y_f = 
for i in range(n):
  y_f += c[i][0]*basicFunction(i, y)
  
y_f

In [ ]:
6*x**2 + 1